# Custom Metrics collector
This notebook demonstrates a way to tap into the metrics collection at run-time
and draw a graph from the collected metrics.

This notebook requires matplotlib to be installed.

In [ ]:
from io import StringIO
import json
import matplotlib.pyplot as plt
from supplychain_simulation import SupplyChain, Node, Sales, LeadTime, Edge, Simulator
from supplychain_simulation.node import Orders, Stock
from supplychain_simulation.pipeline import Pipeline, Receipt
from supplychain_simulation.strategy import RSQ, Fractional
from supplychain_simulation.types import MetricEntryType

In [ ]:
# Build a supply-chain with 10 periods of sales
supply_chain = SupplyChain(
        nodes=[
            Node(
                "A",
                data=dict(
                    order_quantity=30,
                    reorder_level=25,
                    review_time=1,
                    safety_stock=1,
                ),
                sales=Sales({1: [1], 2: [4], 3: [10], 4: [12], 5: [10], 6: [11], 7: [8], 8: [10], 9: [10], 10: [10]}),
                lead_time=LeadTime(default=1),
                stock=Stock({"A": 70, "C": 5}),
                pipeline=Pipeline([Receipt(sku_code="D", eta=0, quantity=35)]),
            ),
            Node(
                "B",
                data=dict(
                    order_quantity=25,
                    reorder_level=40,
                    review_time=1,
                ),
                backorders=5,
                sales=Sales({1: [15], 2: [15], 3: [15], 4: [14], 5: [2], 6: [17], 7: [18], 8: [10], 9: [12], 10: [15]}),
                lead_time=LeadTime(default=2),
                stock=Stock({"B": 30}),
                pipeline=Pipeline([Receipt(sku_code="D", eta=1, quantity=75)]),
            ),
            Node(
                "C",
                data=dict(
                    order_quantity=150,
                    reorder_level=20,
                    review_time=1,
                ),
                lead_time=LeadTime({1: 3, 2: 7}, default=3),
                stock=Stock({"C": 200}),
            ),
            Node(
                "D",
                data=dict(
                    order_quantity=200,
                    reorder_level=20,
                    review_time=2,
                ),
                lead_time=LeadTime(default=4),
                orders=Orders({"B": 15}),
                stock=Stock({"D": 40}),
            ),
        ],
        edges=[
            Edge(source="C", destination="A", number=2),
            Edge(source="D", destination="A", number=1),
            Edge(source="D", destination="B", number=3),
        ],
    )

In [ ]:
sim = Simulator(
    supply_chain,
    control_strategy=RSQ(supply_chain),
    release_strategy=Fractional(),
)

## Collector
The metrics are emitted to the `stream.write()` passed to `Simulator.run(stream)`.

Each entry is a JSON serialized string of `MetricEntryType`.

In [ ]:
class MetricsCollector(StringIO):
    """Custom metrics stream
    
    Each metrics entry is a JSON string with the node, event and quantity
    This collector aggregates the sales and backorders for each period
    """
    def __init__(self):
        super().__init__()
        self.periods = []
        self.sales = []
        self.backorders = []
        self.service = []
        
    def write(self, __s: str) -> int:
        """Parse the entry and collect the sales and backorders"""
        log: MetricEntryType = json.loads(__s)
        if log["event"] in ["sales", "backorders"]:
            # get the period of this entry
            period = int(log["period"])
            if period not in self.periods:
                # Add the period to the periods list
                self.periods.append(period)
                self.periods[:] = sorted(self.periods)
                # ensure all lists are of the same length as periods
                self.sales.append(0)
                self.backorders.append(0)
                self.service.append(0)
            # get the list index of the current period
            indx = self.periods.index(period)
            # Add the quantity for each event
            if log["event"] == "sales":
                self.sales[indx] += int(log["quantity"])
            elif log["event"] == "backorders":
                self.backorders[indx] += int(log["quantity"])
            self.service[indx] = self.sales[indx] / (self.sales[indx] + self.backorders[indx]) * 100

In [ ]:
buffer = MetricsCollector()
sim.run(10, stream=buffer)

In [ ]:
fig, ax = plt.subplots()

sales_line, = ax.plot(buffer.periods, buffer.sales, label="sales")
backorders_line, = ax.plot(buffer.periods, buffer.backorders, label="backorders", color='tab:red')
ax2 = ax.twinx()
ax2.set(ylim=(0, 102))
service_line, = ax2.plot(buffer.periods, buffer.service, label="service", color='tab:green')

ax2.set_ylabel('Service Level [%]', color='tab:green')

plt.legend(handles=[sales_line, backorders_line], loc='lower right')

plt.grid()
plt.show()